In [6]:
from PIL import Image
import os
import multiresolutionimageinterface as mir
from glob import glob
import numpy as np

In [11]:
tif_path_list = glob("/home/jangting/dataset/Breast/train/*.tif")

min_val = 1000000000000
max_val = 0
min_size = None
max_size = None

shape_0 = []
shape_1 = []
val_sum = []

for tif_path in tif_path_list:
    reader = mir.MultiResolutionImageReader()
    mr_image = reader.open(tif_path)
    
    shape = mr_image.getDimensions()
    
    shape_0.append(shape[0])
    shape_1.append(shape[1])
    
    val = shape[0] * shape[1]
    val_sum.append(val)
    if val > max_val:
        max_val = val
        max_size = shape
    elif val < min_val:
        min_val = val
        min_size = shape
        
print(max_size)
print(min_size)
print(np.average(shape_0))
print(np.average(shape_1))
print(np.average(val_sum))


(97792, 221184)
(97792, 215552)
97792.0
219852.8
21499845017.6


In [4]:
def get_image_arr_from_mr_image(mr_image, level=2):
    ds = mr_image.getLevelDownsample(level)
    image_patch = mr_image.getUCharPatch(int(mr_image_shape[0]/2), int(mr_image_shape[1]/2), int(1920/4), int(1080/4), level)
    return image_patch

In [1]:
reader = mir.MultiResolutionImageReader()
mr_image = reader.open(tif_path)
print(mr_image.getDimensions())
    
level = 0
img_arr = get_image_arr_from_mr_image(mr_image, level)

img = Image.fromarray(img_arr)
img

NameError: name 'mir' is not defined

In [52]:
output_annotation_path = "annotation.tif"

In [42]:
def save_annotation_arr(mr_image, output_annotation_path):
    annotation_list = mir.AnnotationList()
    xml_repository = mir.XmlRepository(annotation_list)

    xml_repository.setSource('camelyon17/centre_0/patient_010_node_4.xml')
    xml_repository.load()
    annotation_mask = mir.AnnotationToMask()
    camelyon17_type_mask = True

    label_map = {'metastases': 1, 'normal': 2} if camelyon17_type_mask else {'_0': 1, '_1': 1, '_2': 0}
    conversion_order = ['metastases', 'normal'] if camelyon17_type_mask else  ['_0', '_1', '_2']
    annotation_mask.convert(annotation_list, output_annotation_path, mr_image.getDimensions(), mr_image.getSpacing(), label_map, conversion_order)
    
save_annotation_arr(mr_image)

In [57]:
mr_annotation_image = reader.open(output_annotation_path)
#print(mr_annotation_image.getDimensions())
    
annotation_img_arr = get_image_arr_from_mr_image(mr_annotation_image)

annotation_img_arr.shape
#annoration_img = Image.fromarray(annotation_img_arr)
#annoration_img

(270, 480, 1)

In [60]:
annotation_img_arr.min()

0